# 深度学习框架 PyTorch Lightning 和医学图像分析框架 MONAI 教程

### PyTorch Lightning

<img alt="Lightning" src="https://pl-public-data.s3.amazonaws.com/assets_lightning/LightningColor.png" width="400px" style="max-width: 100%;">

PyTorch Lightning 是一个轻量级的基于 PyTorch 的高层次模型接口，它提供了一种简单的方式来组织 PyTorch 代码，使得代码更加模块化、可读性更强、可维护性更高。它的设计目标是让研究人员专注于模型的设计，而不是训练过程的实现。

它的核心思想是将训练过程分为 5 个部分：`LightningModule`、`LightningDataModule`、`Trainer`、`Callbacks` 和 `LightningLogger`。

- `LightningModule` 是模型的核心，它包含了模型的定义、前向传播、损失函数、优化器等；
- `LightningDataModule` 是数据的核心，它包含了数据的加载、预处理、划分等；
- `Trainer` 是训练过程的核心，它包含了训练过程的超参数、优化器、学习率调整策略、训练过程的配置等；
- `Callbacks` 是训练过程的钩子函数，它包含了训练过程中的一些回调函数，如模型保存、学习率调整、训练过程可视化等；
- `LightningLogger` 是训练过程的日志记录器，它包含了训练过程中的一些日志记录，如训练过程的可视化、训练过程的日志记录等。

### MONAI

<img src="https://raw.githubusercontent.com/Project-MONAI/MONAI/dev/docs/images/MONAI-logo-color.png" width="400px" style="max-width: 100%;" alt='project-monai'>

MONAI (Medical Open Network for AI) 是一个开源的医学图像分析框架，它旨在提供一个用于医学图像分析的端到端的深度学习框架，以加速 AI 在医学图像分析领域的研究和应用。

MONAI 提供了一系列的工具，如数据加载、数据预处理、数据划分、模型定义、模型训练、模型评估、模型推理等，它们都是基于 PyTorch 实现的，因此可以很方便地与 PyTorch 结合使用。


## PyTorch Lightning

PyTorch Lightning 从 2.0 版本开始更名为 Lightning，文档也从 [PyTorch Lightning](https://pytorch-lightning.readthedocs.io/en/latest/) 更名为 [Lightning](https://lightning.ai/docs/pytorch/latest/)。为了避免混淆，在下文中，我们将 PyTorch Lightning 简称为 Lightning。

### 安装

Lightning 的安装非常简单。

若想用 pip 安装，可以使用以下命令：

```bash
pip install lightning
```

若想用 conda 安装，可以使用以下命令：

```bash
conda install lightning -c conda-forge
```

安装完成后，可以使用以下命令查看版本信息：

```bash
lightning --version
```

### 从 PyTorch 迁移到 Lightning

Lightning 的设计目标是让研究人员专注于模型的设计，而不是训练过程的实现。因此，Lightning 的使用非常简单，只需要将 PyTorch 的代码稍作修改即可。

下面我们以训练一个简单的 MNIST 分类模型为例，来介绍如何从 PyTorch 迁移到 Lightning。

In [1]:
!lightning --version

lightning, version 2.0.3


In [26]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from tqdm import tqdm

# Set the device (CPU or GPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 10)

    def forward(self, x):
        x = x.view(-1, 784)  # Flatten the input tensor
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Define the training parameters
batch_size = 64
learning_rate = 0.01
num_epochs = 10
validation_split = 0.2

# Load the MNIST dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
dataset = datasets.MNIST(root=os.getcwd(), train=True, transform=transform, download=True)

# Split the dataset into training and validation sets
train_size = int((1 - validation_split) * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

test_dataset = datasets.MNIST(root=os.getcwd(), train=False, transform=transform, download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Initialize the model and move it to the device
model = Net().to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    with tqdm(total=len(train_loader), desc=f"Epoch {epoch+1}/{num_epochs}", unit="batch") as pbar:
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * images.size(0)

            # Update the progress bar
            pbar.update(1)
            pbar.set_postfix({"Train Loss": train_loss / ((pbar.n - 1) * train_loader.batch_size + images.size(0))})

    # Validation
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for val_images, val_labels in val_loader:
            val_images, val_labels = val_images.to(device), val_labels.to(device)

            val_outputs = model(val_images)
            val_loss += criterion(val_outputs, val_labels).item()

            _, val_predicted = torch.max(val_outputs.data, 1)
            val_total += val_labels.size(0)
            val_correct += (val_predicted == val_labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss /= len(val_loader)

    # Print validation metrics
    tqdm.write(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_accuracy:.2f}%")

# Test the model
model.eval()
total_correct = 0
total_samples = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)

        total_samples += labels.size(0)
        total_correct += (predicted == labels).sum().item()

# Print the accuracy on the test set
accuracy = 100 * total_correct / total_samples
print(f"Test Accuracy: {accuracy:.2f}%")

# Save the model checkpoint
torch.save(model.state_dict(), "mnist.ckpt")

# Load the model checkpoint
model.load_state_dict(torch.load("mnist.ckpt"))


Epoch 1/10: 100%|██████████| 750/750 [00:05<00:00, 125.41batch/s, Train Loss=1.23]


Validation Loss: 0.5165, Validation Accuracy: 85.27%


Epoch 2/10: 100%|██████████| 750/750 [00:06<00:00, 124.32batch/s, Train Loss=0.426]


Validation Loss: 0.3809, Validation Accuracy: 88.83%


Epoch 3/10: 100%|██████████| 750/750 [00:06<00:00, 124.44batch/s, Train Loss=0.351]


...

Epoch 9/10: 100%|██████████| 750/750 [00:05<00:00, 125.91batch/s, Train Loss=0.228]


Validation Loss: 0.2304, Validation Accuracy: 93.23%


Epoch 10/10: 100%|██████████| 750/750 [00:06<00:00, 123.40batch/s, Train Loss=0.213]


Validation Loss: 0.2172, Validation Accuracy: 93.61%
Test Accuracy: 94.06%


<All keys matched successfully>

从上面的 PyTorch 代码可以看出，即使是一个简单的 MNIST 分类模型，也需要写很多代码，而且代码的可读性和可维护性都不高。下面我们来看看如何用 Lightning 重构这个模型。

### LightningDataModule

首先，我们需要将数据的加载、预处理、划分等操作封装到 `LightningDataModule` 中，这样可以使得数据的加载、预处理、划分等操作更加模块化、可读性更强、可维护性更高。

In [15]:
import lightning as pl
from torchvision import datasets

In [16]:
class MNISTDataModule(pl.LightningDataModule):
    def __init__(self, data_dir: str = './'):
        super().__init__()
        self.data_dir = data_dir
        self.transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])

    def prepare_data(self):
        datasets.MNIST(self.data_dir, train=True, download=True)
        datasets.MNIST(self.data_dir, train=False, download=True)

    def setup(self, stage=None):
        if stage == 'fit' or stage is None:
            mnist_full = datasets.MNIST(self.data_dir, train=True, transform=self.transform)
            self.mnist_train, self.mnist_val = random_split(mnist_full, [55000, 5000])
        if stage == 'test' or stage is None:
            self.mnist_test = datasets.MNIST(self.data_dir, train=False, transform=self.transform)

    def train_dataloader(self):
        mnist_train = DataLoader(self.mnist_train, batch_size=32)
        return mnist_train

    def val_dataloader(self):
        mnist_val = DataLoader(self.mnist_val, batch_size=32)
        return mnist_val

    def test_dataloader(self):
        mnist_test = DataLoader(self.mnist_test, batch_size=32)
        return mnist_test

这样一来，我们就可以将数据的加载、预处理、划分等操作从模型中分离出来，使得模型的定义更加简洁。

### LightningModule

接下来，我们需要将模型的定义、前向传播、损失函数、优化器等操作封装到 `LightningModule` 中，这样可以使得模型的定义、前向传播、损失函数、优化器等操作更加模块化、可读性更强、可维护性更高。

In [ ]:
class MNISTClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(nn.Linear(28 * 28, 64), nn.ReLU(), nn.Linear(64, 3))
        self.decoder = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 28 * 28))
        self.classifier = nn.Sequential(nn.Linear(3, 64), nn.ReLU(), nn.Linear(64, 10))

    def forward(self, x):
        x = x.view(x.size(0), -1)
        z = self.encoder(x)
        y_hat = self.classifier(z)
        return y_hat

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('val_loss', loss)

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        self.log('test_loss', loss)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=5e-3)
        return optimizer